# DSAIT4335 Recommender Systems
# Final Project

In this project, you will work to build different recommendation models and evaluate the effectiveness of these models through offline experiments. The dataset used for the experiments is **MovieLens100K**, a movie recommendation dataset collected by GroupLens: https://grouplens.org/datasets/movielens/100k/. For more details, check the project description on Brightspace.

# Instruction

The MovieLens100K is already splitted into 80% training and 20% test sets. Along with training and test sets, movies metadata as content information is also provided.

**Expected file structure** for this assignment:   
   
   ```
   RecSysProject/
   ├── training.txt
   ├── test.txt
   ├── movies.txt
   └── codes.ipynb
   ```

**Note:** Be sure to run all cells in each section sequentially, so that intermediate variables and packages are properly carried over to subsequent cells.

**Note** Be sure to run all cells such that the submitted file contains the output of each cell.

**Note** Feel free to add cells if you need more for answering a question.

**Submission:** Answer all the questions in this jupyter-notebook file. Submit this jupyter-notebook file (your answers included) to Brightspace. Change the name of this jupyter-notebook file to your group number: example, group10 -> 10.ipynb.

# Setup

In [ ]:
%pip install transformers torch  # For BERT
%pip install -r requirements.txt
# you can refer https://huggingface.co/docs/transformers/en/model_doc/bert for various versions of the pre-trained model BERT

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
from scipy.spatial.distance import cosine, correlation
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from transformers import logging 
from recommendation_algorithms.hybrid_recommender import HybridRecommender
from recommendation_algorithms.matrix_factorization import MatrixFactorizationSGD
from recommendation_algorithms.bayesian_probabilistic_ranking import BayesianProbabilisticRanking
from recommendation_algorithms.item_knn import ItemKNN
from recommendation_algorithms.user_knn import UserKNN
from recommendation_algorithms.content_based import ContentBasedRecommender
from evaluation.grid_search import grid_search
from evaluation.score_prediction_metrics import MAE, MSE, RMSE 
logging.set_verbosity_error()
import re
import time, math
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(10)

print("Libraries imported successfully!")

# Load dataset

In [ ]:
# loading the training set and test set
columns_name=['user_id','item_id','rating','timestamp']
train_data = pd.read_csv('data/training.txt', sep='\t', names=columns_name)
test_data = pd.read_csv('data/test.txt', sep='\t', names=columns_name)

display(train_data[['user_id','item_id','rating']].head())
print(f'The shape of the training data: {train_data.shape}')
print(f'The shape of the test data: {test_data.shape}')

movies = pd.read_csv('data/movies.txt',names=['item_id','title','genres','description'],sep='\t')
display(movies.head())

# Task 1) Implementation of different recommendation models as well as a hybrid model combining those recommendation models

In [ ]:
percentage = 0.1
movies_small = movies.iloc[0: int(percentage * len(movies))]
train_data_small = train_data[train_data["item_id"].isin(movies_small["item_id"])]
content = movies_small["title"] + movies_small["description"] + movies_small["description"]

### Content-Based

<div style="background-color: #19e0d0ff; color:#FFFFFF"> 
Explanation for why our model works.
</div>


In [ ]:
BERT_MODEL_NAME = 'boltuix/bert-mini'
CBR = ContentBasedRecommender(BERT_MODEL_NAME, train_data_small, 16, "weighted_average", content)
CBR.train(train_data_small)

#### Hyperparameter Tuning

In [ ]:
hyperparameters_content_based = {
    "aggregation_method": ["average", "weighted_average", "avg_pos"],
    "bert_model": ['boltuix/bert-mini', 'distilbert-base-uncased'],
    "data": [train_data_small],
    "batch_size": [16],
    "content": [content]
}

best_parameters_cb, params_cb = grid_search(hyperparameters_content_based, ContentBasedRecommender, train_data_small, RMSE)

```
Best params metric 0.4182111704005534
Best params: [('aggregation_method', 'avg_pos'), ('bert_model', 'boltuix/bert-mini'), ('batch_size', 16)]
```

#### Train Best Model

In [ ]:
content_based_best = ContentBasedRecommender(**best_parameters_cb)
content_based_best.train(train_data_small)

#### User-based Collaborative Filtering

In [ ]:
u_knn = UserKNN(2)
u_knn.train(train_data_small)

In [ ]:
u_knn.calculate_all_predictions(train_data_small)
display(u_knn.predictions.head())
u_knn.calculate_all_rankings(5, train_data_small)
display(u_knn.get_ranking(1, 5))

In [ ]:
hyperparameters_user_knn = {
    "k": [2, 3, 5, 7, 9, 11]
}

similarity_matrix = u_knn.similarity_matrix
best_parameters_uknn, params_uknn = grid_search(hyperparameters_user_knn, UserKNN, train_data_small, RMSE, similarity_matrix=similarity_matrix)

#### Train Best Model

In [ ]:
user_knn_best = UserKNN(**best_parameters_uknn)
user_knn_best.train(train_data_small)

### Item-based Collaborative Filtering

In [ ]:
i_knn = ItemKNN(2)
i_knn.train(train_data_small)

In [ ]:
i_knn.calculate_all_predictions(train_data_small)
display(i_knn.predictions.head())
i_knn.calculate_all_rankings(5, train_data_small)
display(i_knn.get_ranking(0, 5))

#### Hyperparameter Tuning

In [ ]:
hyperparameters_item_knn = {
    "k": [2, 3, 5, 7, 9, 11]
}

similarity_matrix = i_knn.similarity_matrix
best_parameters_iknn, params_iknn = grid_search(hyperparameters_item_knn, ItemKNN, train_data_small, RMSE, similarity_matrix=similarity_matrix)

#### Train Best Model

In [ ]:
item_knn_best = ItemKNN(**best_parameters_iknn)
item_knn_best.train(train_data_small)

### Matrix Factorization

In [ ]:
n_factors=20
learning_rate=0.01 
regularization=0.02 
n_epochs=20 
use_bias=True
mf = MatrixFactorizationSGD(n_factors=n_factors, learning_rate=learning_rate, regularization=regularization, n_epochs=n_epochs, use_bias=use_bias)
mf.train(train_data_small)

In [ ]:
mf.calculate_all_predictions(train_data_small)
display(mf.predictions.head())
mf.calculate_all_rankings(5, train_data_small)
display(mf.get_ranking(0, 5))

In [ ]:
hyperparameters_matrix_factorization = {
    'n_factors':[5, 10, 20, 25], 
    'learning_rate':[0.001, 0.01, 0.05, 0.1], 
    'regularization':[0.002, 0.02, 0.2], 
    'n_epochs': [5, 20], 
    'use_bias':[True, False]
}

best_parameters_mf, params_mf = grid_search(hyperparameters_matrix_factorization, MatrixFactorizationSGD, train_data_small, RMSE, similarity_matrix=similarity_matrix)

#### Train Best Model

In [ ]:
mf_best = MatrixFactorizationSGD(**best_parameters_mf)
mf_best.train(train_data_small)

### Bayesian Probabilistic Ranking

In [ ]:
n_factors=20
learning_rate=0.01 
regularization=0.02 
n_epochs=20 
use_bias=True
bpr = BayesianProbabilisticRanking(n_factors=n_factors, learning_rate=learning_rate, regularization=regularization, n_epochs=n_epochs, use_bias=use_bias)
bpr.train(train_data_small)

In [ ]:
bpr.calculate_all_predictions(train_data_small)
display(bpr.predictions.head())
bpr.calculate_all_rankings(5, train_data_small)
display(bpr.get_ranking(0, 5))

#### Hyperparameter Tuning

In [ ]:
hyperparameters_bpr = {
    'n_factors':[5, 10, 20, 25], 
    'learning_rate':[0.001, 0.01, 0.05, 0.1], 
    'regularization':[0.002, 0.02, 0.2], 
    'n_epochs': [5, 20], 
    'use_bias':[True, False]
}

best_parameters_bpr, params_bpr = grid_search(hyperparameters_bpr, BayesianProbabilisticRanking, train_data_small, RMSE, similarity_matrix=similarity_matrix)

#### Train Best Model

In [ ]:
bpr_best = BayesianProbabilisticRanking(**best_parameters_mf)
bpr_best.train(train_data_small)

## Hybrid Recommender

In [ ]:
rating_recommenders = [content_based_best, item_knn_best, user_knn_best, mf_best, bpr_best]
ranking_recommenders = [content_based_best, item_knn_best, user_knn_best, mf_best, bpr_best]
max_k = 10 # Recommendation list size
hybrid_recommender = HybridRecommender(train_data, rating_recommenders, ranking_recommenders, max_k, True)

In [ ]:
user_id = 1
item_id = 2
predicted_score = hybrid_recommender.predict_score(user_id, item_id)
actual_score = train_data.loc[((train_data['user_id'] == user_id) & (train_data['item_id'] == item_id)), 'rating'].values[0]
print(f'Predicted score {predicted_score} for user {user_id} and item {item_id}, actual score: {actual_score}.')

predicted_ranking = hybrid_recommender.predict_ranking(user_id, max_k)
print(f"Predicted ranking for user {user_id}")
for i in range(len(predicted_ranking)):
    print(f"  {i + 1}: {predicted_ranking[i]}")

# Task 2) Experiments for both rating prediction and ranking tasks, and conducting offline evaluation

We are going to use our best recommenders from the previous task:

In [ ]:
for recommender in ranking_recommenders:
    recommender.calculate_all_predictions(train_data)

In [ ]:
ranking_recommenders = [content_based_best, item_knn_best, user_knn_best, mf_best, bpr_best]
rmse_cb = RMSE(train_data["rating"], content_based_best.predictions["predicted_score"])
rmse_iknn = RMSE(train_data["rating"], item_knn_best.predictions["predicted_score"])
rmse_uknn = RMSE(train_data["rating"], user_knn_best.predictions["predicted_score"])
rmse_mf = RMSE(train_data["rating"], mf_best.predictions["predicted_score"])
rmse_bpr = RMSE(train_data["rating"], bpr_best.predictions["predicted_score"])

In [ ]:
print("RMSE Best Content-Based:", rmse_cb)
print("RMSE Best ItemKNN:", rmse_iknn)
print("RMSE Best UserKNN:", rmse_uknn)
print("RMSE Best Matrix Factorization:", rmse_mf)
print("RMSE Best BPR:", rmse_bpr)

# Task 3) Implement baselines for both rating prediction and ranking tasks, and perform experiments with those baselines

# Task 4) Analysis of recommendation models. Analyzing the coefficients of hybrid model and the success of recommendation models for different users' groups. 

# Task 5) Evaluation of beyond accuracy